In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [91]:
mg = RasterModelGrid((20,30),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
#d_h = mg.add_ones('link', 'hydraulic__diameter')

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)

h = mg.add_zeros('node', 'hydraulic__head')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

#add input to node 310
moulin_Q=1.
mg.at_node['input__discharge'][51]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
Q[mg.active_links] = 1.
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print "Number of links = ", mg.number_of_links
print "Number of nodes = ", mg.number_of_nodes
print "Number of active links = ", mg.number_of_active_links
print "Number of core nodes = ", mg.number_of_core_nodes

#code added for fsolve algorithm
dhdx = mg.add_zeros('link', 'hydraulic_head_gradient', noclobber=False)
net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)


pfn = PresFlowNetwork(mg)
pfn.run_one_step()
mc = MeltCreep(mg, dt=500.)

Number of links =  247
Number of nodes =  150
Number of active links =  48
Number of core nodes =  48
Number of iterations = 1 tolerance = 0.0


In [82]:
mg.active_links

array([10, 11, 12])

In [92]:
mg.at_link['conduit__discharge'][mg.active_links]

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [76]:
mc.run_one_step()
pfn.run_one_step()


mean ddh =  0.00602016820239
Number of iterations = 1 tolerance = 1.18423789293e-15


In [48]:
pfn.run_one_step()

Number of iterations = 1 tolerance = 0.042972532952
Number of iterations = 2 tolerance = 0.0113919126592
Number of iterations = 3 tolerance = 0.000913160009722


In [17]:
from scipy.optimize import fsolve
def network_residuals(heads):
    h[mg.core_nodes] = heads
    mg.calc_grad_at_link(h, out=dhdx)
    Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    return mg.calc_net_flux_at_node(Q, out=net_node_flux)[mg.core_nodes] + mg.at_node['input__discharge'][mg.core_nodes]
#This function doesn't account for inflow at nodes


In [39]:
mc.run_one_step()
mg.calc_grad_at_link(h, out=dhdx)
mg.at_node['hydraulic__head'][mg.core_nodes] = fsolve(network_residuals, h[mg.core_nodes])
mg.at_link['conduit__discharge'][mg.core_nodes] = np.sign(dhdx[mg.core_nodes])*np.sqrt(np.fabs(dhdx[mg.core_nodes]))

print "mean h =", mean(mg.at_node['hydraulic__head'][mg.core_nodes] )
print "mean Q =", mean(Q)

mean ddh =  -8.29966586555e+56
mean h = 2.89426692424
mean Q = 0.169208244866


NameError: name 'eqh' is not defined

In [78]:
figure()
imshow_grid_at_node(mg, h)

In [5]:
def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='jet', magnitude= False, lw=2):
    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    for i, value in enumerate(values):
        xs = [link_head_x[i],link_tail_x[i]]
        ys = [link_head_y[i],link_tail_y[i]]
        img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
    colorbar(scalarMap)


In [24]:
figure()
plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
print mean(mg.at_link['hydraulic__diameter'])

3.50074727364e+69


In [56]:
figure()
plot_links(mg, 'conduit__discharge', autoscale=True, magnitude=True)
#print mean(mg.at_link['hydraulic__diameter'])

In [94]:
nsteps = 1000
step_start = 1000
fig1, axs = subplots(2,2,figsize=(9,6))
every = 10
for step in arange(nsteps)+step_start:
    print "step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean()
    pfn.run_one_step()    
    #mg.calc_grad_at_link(h, out=dhdx)
    #h = fsolve(network_residuals, h[mg.core_nodes])
    #Q = np.sign(dhdx)*np.sqrt(np.fabs(dhdx))
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(2,2,1)
        imshow_grid_at_node(mg, h)
        #plot(mg.at_node['hydraulic__head'][mg.core_nodes])
        ylim([300,400])
        subplot(2,2,2)
        plot_links(mg, 'conduit__discharge', vmin=0., vmax=1., magnitude=True, autoscale=False)
        #plot(mg.at_link['conduit__discharge'][mg.active_links])
        #ylabel('')
        ylim([0,1.5])
        subplot(2,2,3)
        plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
        subplot(2,2,4)
        plot(mg.at_link['hydraulic__diameter'][mg.active_links])
        ylim([0.5,1.5])
        image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
        tight_layout()
        savefig(image_name)
        fig1.clf()

#im_ani = animation.ArtistAnimation(fig1,ims)

step = 1000  avg d_h= 0.36957896381
Number of iterations = 1 tolerance = 9.10845473119e-14
mean ddh =  6.12357082029e-05
step = 1001  avg d_h= 0.369590863866
Number of iterations = 1 tolerance = 1.51853067158e-13
mean ddh =  6.11317407319e-05
step = 1002  avg d_h= 0.369602743719
Number of iterations = 1 tolerance = 5.65982446095e-14
mean ddh =  6.10279947451e-05
step = 1003  avg d_h= 0.36961460341
Number of iterations = 1 tolerance = 4.03126606171e-14
mean ddh =  6.09244696414e-05
step = 1004  avg d_h= 0.369626442983
Number of iterations = 1 tolerance = 7.11548875376e-14
mean ddh =  6.08211648221e-05
step = 1005  avg d_h= 0.369638262481
Number of iterations = 1 tolerance = 5.22174895915e-14
mean ddh =  6.07180796906e-05
step = 1006  avg d_h= 0.369650061946
Number of iterations = 1 tolerance = 5.88360378935e-14
mean ddh =  6.06152136522e-05
step = 1007  avg d_h= 0.36966184142
Number of iterations = 1 tolerance = 3.47095037897e-14
mean ddh =  6.05125661144e-05
step = 1008  avg d_h= 0.369

step = 1071  avg d_h= 0.370376012009
Number of iterations = 1 tolerance = 3.13036633652e-14
mean ddh =  5.43719821272e-05
step = 1072  avg d_h= 0.370386578224
Number of iterations = 1 tolerance = 1.76487296999e-13
mean ddh =  5.42823580606e-05
step = 1073  avg d_h= 0.370397127023
Number of iterations = 1 tolerance = 4.2287932415e-14
mean ddh =  5.41929175313e-05
step = 1074  avg d_h= 0.37040765844
Number of iterations = 1 tolerance = 3.94915581718e-14
mean ddh =  5.41036600669e-05
step = 1075  avg d_h= 0.370418172511
Number of iterations = 1 tolerance = 4.16923440216e-14
mean ddh =  5.40145851961e-05
step = 1076  avg d_h= 0.370428669273
Number of iterations = 1 tolerance = 2.70859723539e-14
mean ddh =  5.39256924493e-05
step = 1077  avg d_h= 0.37043914876
Number of iterations = 1 tolerance = 4.89492705628e-14
mean ddh =  5.38369813585e-05
step = 1078  avg d_h= 0.370449611007
Number of iterations = 1 tolerance = 2.83234084334e-14
mean ddh =  5.37484514571e-05
step = 1079  avg d_h= 0.370

step = 1141  avg d_h= 0.371075457118
Number of iterations = 1 tolerance = 1.09919018332e-13
mean ddh =  4.85170085716e-05
step = 1142  avg d_h= 0.371084885524
Number of iterations = 1 tolerance = 1.07064819973e-13
mean ddh =  4.84391696756e-05
step = 1143  avg d_h= 0.371094298804
Number of iterations = 1 tolerance = 2.49641742463e-13
mean ddh =  4.83614846388e-05
step = 1144  avg d_h= 0.371103696987
Number of iterations = 1 tolerance = 3.40338868199e-13
mean ddh =  4.82839530839e-05
step = 1145  avg d_h= 0.371113080104
Number of iterations = 1 tolerance = 2.2690298714e-13
mean ddh =  4.82065746327e-05
step = 1146  avg d_h= 0.371122448183
Number of iterations = 1 tolerance = 1.33557516898e-13
mean ddh =  4.812934891e-05
step = 1147  avg d_h= 0.371131801255
Number of iterations = 1 tolerance = 9.88364482849e-14
mean ddh =  4.80522755407e-05
step = 1148  avg d_h= 0.371141139349
Number of iterations = 1 tolerance = 5.434773002e-14
mean ddh =  4.79753541513e-05
step = 1149  avg d_h= 0.37115

step = 1211  avg d_h= 0.37170048682
Number of iterations = 1 tolerance = 5.42216734475e-14
mean ddh =  4.34201695641e-05
step = 1212  avg d_h= 0.371708924748
Number of iterations = 1 tolerance = 4.51768252437e-14
mean ddh =  4.33522473841e-05
step = 1213  avg d_h= 0.371717349476
Number of iterations = 1 tolerance = 1.04700970116e-13
mean ddh =  4.32844551781e-05
step = 1214  avg d_h= 0.37172576103
Number of iterations = 1 tolerance = 1.7029433419e-13
mean ddh =  4.32167926394e-05
step = 1215  avg d_h= 0.371734159435
Number of iterations = 1 tolerance = 1.35920210272e-13
mean ddh =  4.31492594631e-05
step = 1216  avg d_h= 0.371742544717
Number of iterations = 1 tolerance = 1.77089824286e-13
mean ddh =  4.30818553446e-05
step = 1217  avg d_h= 0.371750916899
Number of iterations = 1 tolerance = 1.57326697966e-13
mean ddh =  4.30145799799e-05
step = 1218  avg d_h= 0.371759276008
Number of iterations = 1 tolerance = 1.32002048181e-13
mean ddh =  4.29474330665e-05
step = 1219  avg d_h= 0.371

step = 1281  avg d_h= 0.372260595165
Number of iterations = 1 tolerance = 2.41288470685e-14
mean ddh =  3.89635229696e-05
step = 1282  avg d_h= 0.372268167024
Number of iterations = 1 tolerance = 4.43268107405e-14
mean ddh =  3.89040055889e-05
step = 1283  avg d_h= 0.372275727317
Number of iterations = 1 tolerance = 2.9557375066e-14
mean ddh =  3.8844598744e-05
step = 1284  avg d_h= 0.372283276065
Number of iterations = 1 tolerance = 1.29723778016e-13
mean ddh =  3.87853021842e-05
step = 1285  avg d_h= 0.372290813289
Number of iterations = 1 tolerance = 1.968992099e-13
mean ddh =  3.87261156593e-05
step = 1286  avg d_h= 0.372298339012
Number of iterations = 1 tolerance = 1.52914717925e-13
mean ddh =  3.86670389206e-05
step = 1287  avg d_h= 0.372305853255
Number of iterations = 1 tolerance = 8.39791199544e-14
mean ddh =  3.86080717193e-05
step = 1288  avg d_h= 0.372313356038
Number of iterations = 1 tolerance = 3.93042080364e-14
mean ddh =  3.85492138077e-05
step = 1289  avg d_h= 0.3723

...

step = 1701  avg d_h= 0.374635882926
Number of iterations = 1 tolerance = 1.5688954765e-13
mean ddh =  2.13157641881e-05
step = 1702  avg d_h= 0.374640025261
Number of iterations = 1 tolerance = 1.577430316e-13
mean ddh =  2.12869208182e-05
step = 1703  avg d_h= 0.374644161991
Number of iterations = 1 tolerance = 1.57582280558e-13
mean ddh =  2.1258123859e-05
step = 1704  avg d_h= 0.374648293124
Number of iterations = 1 tolerance = 4.4638482725e-13
mean ddh =  2.1229373223e-05
step = 1705  avg d_h= 0.37465241867
Number of iterations = 1 tolerance = 9.32888026087e-14
mean ddh =  2.12006688214e-05
step = 1706  avg d_h= 0.374656538638
Number of iterations = 1 tolerance = 2.17019689256e-13
mean ddh =  2.11720105667e-05
step = 1707  avg d_h= 0.374660653037
Number of iterations = 1 tolerance = 1.76180829185e-13
mean ddh =  2.11433983719e-05
step = 1708  avg d_h= 0.374664761876
Number of iterations = 1 tolerance = 1.82128617743e-13
mean ddh =  2.1114832149e-05
step = 1709  avg d_h= 0.37466886

step = 1771  avg d_h= 0.374912788283
Number of iterations = 1 tolerance = 6.83342271657e-14
mean ddh =  1.94041620716e-05
step = 1772  avg d_h= 0.374916559133
Number of iterations = 1 tolerance = 8.21322176936e-14
mean ddh =  1.93783650123e-05
step = 1773  avg d_h= 0.374920324969
Number of iterations = 1 tolerance = 1.16366407251e-13
mean ddh =  1.93526086451e-05
step = 1774  avg d_h= 0.3749240858
Number of iterations = 1 tolerance = 3.88080771222e-14
mean ddh =  1.93268928948e-05
step = 1775  avg d_h= 0.374927841633
Number of iterations = 1 tolerance = 5.72088672731e-14
mean ddh =  1.9301217686e-05
step = 1776  avg d_h= 0.374931592477
Number of iterations = 1 tolerance = 1.03500541471e-13
mean ddh =  1.92755829436e-05
step = 1777  avg d_h= 0.374935338339
Number of iterations = 1 tolerance = 5.14264556865e-14
mean ddh =  1.92499885924e-05
step = 1778  avg d_h= 0.374939079228
Number of iterations = 1 tolerance = 2.10564204961e-13
mean ddh =  1.92244345575e-05
step = 1779  avg d_h= 0.374

step = 1841  avg d_h= 0.375165060671
Number of iterations = 1 tolerance = 2.25185610899e-13
mean ddh =  1.76926532413e-05
step = 1842  avg d_h= 0.37516849892
Number of iterations = 1 tolerance = 4.36803371251e-14
mean ddh =  1.76695309576e-05
step = 1843  avg d_h= 0.375171932675
Number of iterations = 1 tolerance = 8.93810488585e-14
mean ddh =  1.76464444566e-05
step = 1844  avg d_h= 0.375175361943
Number of iterations = 1 tolerance = 2.28902545067e-14
mean ddh =  1.76233936735e-05
step = 1845  avg d_h= 0.375178786732
Number of iterations = 1 tolerance = 3.51304633532e-14
mean ddh =  1.76003785434e-05
step = 1846  avg d_h= 0.375182207049
Number of iterations = 1 tolerance = 3.12631864841e-14
mean ddh =  1.75773990017e-05
step = 1847  avg d_h= 0.375185622899
Number of iterations = 1 tolerance = 9.69409737668e-14
mean ddh =  1.75544549839e-05
step = 1848  avg d_h= 0.375189034292
Number of iterations = 1 tolerance = 8.14498931264e-14
mean ddh =  1.75315464255e-05
step = 1849  avg d_h= 0.3

step = 1911  avg d_h= 0.375395255891
Number of iterations = 1 tolerance = 2.84088724767e-13
mean ddh =  1.61570757547e-05
step = 1912  avg d_h= 0.375398395727
Number of iterations = 1 tolerance = 1.69158668554e-13
mean ddh =  1.61363086483e-05
step = 1913  avg d_h= 0.375401531528
Number of iterations = 1 tolerance = 9.22780370634e-14
mean ddh =  1.61155730929e-05
step = 1914  avg d_h= 0.375404663299
Number of iterations = 1 tolerance = 1.20690494635e-13
mean ddh =  1.60948690323e-05
step = 1915  avg d_h= 0.375407791047
Number of iterations = 1 tolerance = 1.20260283213e-13
mean ddh =  1.60741964107e-05
step = 1916  avg d_h= 0.375410914778
Number of iterations = 1 tolerance = 1.08079054965e-13
mean ddh =  1.60535551719e-05
step = 1917  avg d_h= 0.375414034497
Number of iterations = 1 tolerance = 2.09546500522e-13
mean ddh =  1.60329452603e-05
step = 1918  avg d_h= 0.375417150211
Number of iterations = 1 tolerance = 4.02172508259e-13
mean ddh =  1.60123666206e-05
step = 1919  avg d_h= 0.

step = 1981  avg d_h= 0.375605624106
Number of iterations = 1 tolerance = 1.33900992146e-13
mean ddh =  1.47766062434e-05
step = 1982  avg d_h= 0.375608495673
Number of iterations = 1 tolerance = 1.04075313182e-13
mean ddh =  1.47579183349e-05
step = 1983  avg d_h= 0.375611363609
Number of iterations = 1 tolerance = 1.66754341816e-13
mean ddh =  1.47392583159e-05
step = 1984  avg d_h= 0.375614227918
Number of iterations = 1 tolerance = 1.79025775685e-13
mean ddh =  1.4720626138e-05
step = 1985  avg d_h= 0.375617088606
Number of iterations = 1 tolerance = 7.01556868155e-14
mean ddh =  1.47020217528e-05
step = 1986  avg d_h= 0.375619945679
Number of iterations = 1 tolerance = 3.81410181216e-13
mean ddh =  1.46834451121e-05
step = 1987  avg d_h= 0.375622799142
Number of iterations = 1 tolerance = 2.42891355177e-13
mean ddh =  1.46648961668e-05
step = 1988  avg d_h= 0.375625649001
Number of iterations = 1 tolerance = 1.33504318711e-13
mean ddh =  1.46463748694e-05
step = 1989  avg d_h= 0.3

In [15]:
mc = MeltCreep(mg, dt=5000.)
#mean(mc.Q)

In [17]:
plot(mg.at_link['hydraulic__diameter'][200:300])


Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902,
                   'y_coord': 1.0513915948896004},
 'node': {'ID': 1,
          'column': 1,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 62.235756386751902


Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.648267738121763,
                   'y_coord': 0.84814187582238287},
 'node': {'ID': 0,
          'column': 0,
          'row': 0}}

Grid query results:

{'grid location': {'x_coord': 19.6482677381

In [23]:
#figure()

pfn.run_one_step()
mc.run_one_step()

Number of iterations = 1 tolerance = 0.00299676026854
Number of iterations = 2 tolerance = 1.34229231785e-05
mean Q= 0.0179517502025


In [41]:
#for m in mc.melt:
#    print m
print max(mc.melt[~np.isnan(mc.melt)])
print max(mc.creep)
#print mc.d_h.mean()
#print mg.at_link['hydraulic__diameter']

3.35041761374e-06
2.12710591355e-06


In [15]:
#mc.run_one_step()

mc._grid.at_link['conduit__discharge']

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])